In [1]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [2]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [3]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [4]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [5]:
barbMetrics = loadMetricData('barbComp', "metrics")
barbMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-02 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-02 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-02 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-02 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-02 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
barbMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [7]:
# clean metric data

barbMetrics.timestamp = pd.to_datetime(barbMetrics.timestamp)

barbMetrics.heartrate = barbMetrics.heartrate.fillna(0)
barbMetrics.steps = barbMetrics.steps.fillna(0)
barbMetrics.calories = barbMetrics.calories.fillna(0)
barbMetrics.gsr = barbMetrics.gsr.fillna(0)
barbMetrics.skintemp = barbMetrics.skintemp.fillna(0)
barbMetrics.airtemp = barbMetrics.airtemp.fillna(0)

barbMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-04-26 00:00:00,0,0,0,0,0,0
1,2016-04-26 00:01:00,0,0,0,0,0,0
2,2016-04-26 00:02:00,0,0,0,0,0,0
3,2016-04-26 00:03:00,0,0,0,0,0,0
4,2016-04-26 00:04:00,0,0,0,0,0,0


In [8]:
barbActivities = loadMetricData('barbComp', "activities")
barbActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-02 11:58:04,2016-05-02T11:58:04Z,America/New_York,-240,2016-05-02 12:02:15,2016-05-02T12:02:15Z,America/New_York,-240,walk,251,433,23.000000,4.183333,112.243027,NaN,NaN,complete,1,walk__2016-05-02T115804
1,2016-05-02 12:05:02,2016-05-02T12:05:02Z,America/New_York,-240,2016-05-02 12:17:42,2016-05-02T12:17:42Z,America/New_York,-240,walk,646,1152,71.300003,12.666667,114.549538,NaN,NaN,complete,1,walk__2016-05-02T120502
2,2016-05-02 13:01:23,2016-05-02T13:01:23Z,America/New_York,-240,2016-05-02 13:10:46,2016-05-02T13:10:46Z,America/New_York,-240,walk,307,485,18.600000,9.383333,102.285713,NaN,NaN,complete,1,walk__2016-05-02T130123
3,2016-05-02 16:13:28,2016-05-02T16:13:28Z,America/New_York,-240,2016-05-02 16:23:46,2016-05-02T16:23:46Z,America/New_York,-240,walk,272,486,31.100000,10.300000,121.375000,NaN,NaN,complete,1,walk__2016-05-02T161328
4,2016-05-02 16:47:41,2016-05-02T16:47:41Z,America/New_York,-240,2016-05-02 16:53:08,2016-05-02T16:53:08Z,America/New_York,-240,walk,233,435,26.299999,5.450000,116.309013,NaN,NaN,complete,1,walk__2016-05-02T164741


In [9]:
barbSleep = loadMetricData('barbComp', "sleep")
barbSleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-04-29 22:38:22,2016-04-29T22:38:22Z,America/New_York,-240,2016-04-29 23:11:22,2016-04-29T23:11:22Z,America/New_York,-240,32,0,...,0,sleep,1980,41.599998,77.544754,NaN,NaN,complete,2,sleep__2016-04-29T223822
1,2016-04-30 04:54:42,2016-04-30T04:54:42Z,America/New_York,-240,2016-04-30 13:54:42,2016-04-30T13:54:42Z,America/New_York,-240,275,139,...,27,sleep,32400,622.700012,67.379333,NaN,NaN,complete,2,sleep__2016-04-30T045442
0,2016-05-01 17:35:39,2016-05-01T17:35:39Z,America/New_York,-240,2016-05-01 18:13:39,2016-05-01T18:13:39Z,America/New_York,-240,37,0,...,2,sleep,2280,43.700001,59.494274,NaN,NaN,complete,2,sleep__2016-05-01T173539
1,2016-05-02 01:57:39,2016-05-02T01:57:39Z,America/New_York,-240,2016-05-02 03:09:39,2016-05-02T03:09:39Z,America/New_York,-240,71,0,...,4,sleep,4320,89.099998,79.341530,NaN,NaN,complete,2,sleep__2016-05-02T015739
2,2016-05-02 03:58:39,2016-05-02T03:58:39Z,America/New_York,-240,2016-05-02 10:40:39,2016-05-02T10:40:39Z,America/New_York,-240,210,103,...,18,sleep,23760,460.600006,58.690727,NaN,NaN,complete,2,sleep__2016-05-02T035839


In [10]:
barbSleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [11]:
barbJournal = loadMetricData('barbComp', "journal")
barbJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
113,5/2/16 7:15,5/2/16 8:15,5/2/16,7:15 AM,8:15 AM,Train Commute,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
114,5/2/16 8:15,5/2/16 9:30,5/2/16,8:15 AM,9:30 AM,setting up computers,work,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN
115,5/2/16 9:30,5/2/16 11:13,5/2/16,9:30 AM,11:13 AM,Class and setting up computers,work,NaN,NaN,NaN,NaN,NaN,6,6,NaN,NaN,NaN,NaN
116,5/2/16 11:13,5/2/16 12:00,5/2/16,11:13 AM,12:00 PM,Work,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,5/2/16 12:00,5/2/16 13:18,5/2/16,12:00 PM,1:18 PM,meeting with anne,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
barbJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative    float64
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [13]:
# data= barbJournal.iloc[:0]
# clean metric data
barbJournal["starttime"]= pd.to_datetime(barbJournal.starttime)

# data= barbJournal.iloc[:1]
barbJournal["endtime"]= pd.to_datetime(barbJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

barbJournal.qualitative = barbJournal.qualitative.fillna(0)
barbJournal.excited = barbJournal.excited.fillna(0)
barbJournal.happy = barbJournal.happy.fillna(0)
barbJournal.calm = barbJournal.calm.fillna(0)
barbJournal.anxious = barbJournal.anxious.fillna(0)
barbJournal.sad = barbJournal.sad.fillna(0)
barbJournal.angry = barbJournal.angry.fillna(0)
barbJournal.hungry = barbJournal.hungry.fillna(0)
barbJournal.tired = barbJournal.tired.fillna(0)
barbJournal.bored = barbJournal.bored.fillna(0)

print(barbJournal.shape)
barbJournal.head(5)
# barbJournal.tail(5)

(118, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,2,6,0,0,0,0,2,1
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,2,6,0,0,0,0,2,0
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,0,6,0,0,0,0,4,0
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,0,6,0,0,0,0,4,0
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,0,6,0,0,0,0,4,0


In [14]:

barbJournal['boolexcited'] =  barbJournal['excited'] > 0
barbJournal['boolhappy'] =  barbJournal['happy'] > 0
barbJournal['boolcalm'] =  barbJournal['calm'] > 0
barbJournal['boolanxious'] =  barbJournal['anxious'] > 0
barbJournal['boolsad'] =  barbJournal['sad'] > 0
barbJournal['boolangry'] =  barbJournal['angry'] > 0
barbJournal['booltired'] =  barbJournal['tired'] > 0
barbJournal['boolhungry'] =  barbJournal['hungry'] > 0
barbJournal['boolbored'] =  barbJournal['bored'] > 0


for i in range(0, barbJournal.shape[0]):
#     barbJournal['intqual'] = int(barbJournal.iloc[i, 8])
    barbJournal['intexcited'] = int(barbJournal.iloc[i, 9])
    barbJournal['inthappy'] = int(barbJournal.iloc[i, 10]) 
    barbJournal['intcalm'] = int(barbJournal.iloc[i, 11])
    barbJournal['intanxious'] = int(barbJournal.iloc[i, 12])
    barbJournal['intsad'] = int(barbJournal.iloc[i, 13]) 
    barbJournal['intangry'] = int(barbJournal.iloc[i, 14]) 
    barbJournal['inttired'] = int(barbJournal.iloc[i, 15]) 
    barbJournal['inthungry'] = int(barbJournal.iloc[i, 16]) 
    barbJournal['intbored'] = int(barbJournal.iloc[i, 17]) 
    
barbJournal['exercise'] = barbJournal['activity'].str.contains('exercise')
barbJournal['code'] = barbJournal['activity'].str.contains('code')
barbJournal['anne'] = barbJournal['activity'].str.contains('anne')
barbJournal['will'] = barbJournal['activity'].str.contains('will')
barbJournal['mick'] = barbJournal['activity'].str.contains('mick')
barbJournal['geyer'] = barbJournal['activity'].str.contains('geyer')

barbJournal['home'] = barbJournal['location'].str.contains('home')
barbJournal['work'] = barbJournal['location'].str.contains("work")
barbJournal['school'] = barbJournal['location'].str.contains("school")




boolexcited = barbJournal.iloc[:, 18].values.reshape(barbJournal.shape[0], 1)
boolhappy = barbJournal.iloc[:, 19].values.reshape(barbJournal.shape[0], 1)
boolcalm = barbJournal.iloc[:, 20].values.reshape(barbJournal.shape[0], 1)
boolanxious = barbJournal.iloc[:, 21].values.reshape(barbJournal.shape[0], 1)
boolsad = barbJournal.iloc[:, 22].values.reshape(barbJournal.shape[0], 1)
boolangry = barbJournal.iloc[:, 23].values.reshape(barbJournal.shape[0], 1)
booltired = barbJournal.iloc[:, 24].values.reshape(barbJournal.shape[0], 1)
boolhungry = barbJournal.iloc[:, 25].values.reshape(barbJournal.shape[0], 1)
boolbored = barbJournal.iloc[:, 26].values.reshape(barbJournal.shape[0], 1)
intexcited = barbJournal.iloc[:, 27].values.reshape(barbJournal.shape[0], 1)
inthappy = barbJournal.iloc[:, 28].values.reshape(barbJournal.shape[0], 1)
intcalm = barbJournal.iloc[:, 29].values.reshape(barbJournal.shape[0], 1)
intanxious = barbJournal.iloc[:, 30].values.reshape(barbJournal.shape[0], 1)
intsad = barbJournal.iloc[:, 31].values.reshape(barbJournal.shape[0], 1)
intangry = barbJournal.iloc[:, 32].values.reshape(barbJournal.shape[0], 1)
inttired = barbJournal.iloc[:, 33].values.reshape(barbJournal.shape[0], 1)
inthungry = barbJournal.iloc[:, 34].values.reshape(barbJournal.shape[0], 1)
intbored = barbJournal.iloc[:, 35].values.reshape(barbJournal.shape[0], 1)
exercise = barbJournal.iloc[:, 36].values.reshape(barbJournal.shape[0], 1)
code = barbJournal.iloc[:, 37].values.reshape(barbJournal.shape[0], 1)
anne = barbJournal.iloc[:, 38].values.reshape(barbJournal.shape[0], 1)
will = barbJournal.iloc[:, 39].values.reshape(barbJournal.shape[0], 1)
mick = barbJournal.iloc[:, 40].values.reshape(barbJournal.shape[0], 1)
geyer = barbJournal.iloc[:, 41].values.reshape(barbJournal.shape[0], 1)
home = barbJournal.iloc[:, 42].values.reshape(barbJournal.shape[0], 1)
work = barbJournal.iloc[:, 43].values.reshape(barbJournal.shape[0], 1)
school = barbJournal.iloc[:, 44].values.reshape(barbJournal.shape[0], 1)

barbJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,intbored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,...,0,False,False,False,False,False,False,False,True,False
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,...,0,False,False,False,False,True,False,False,True,False
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,...,0,False,False,False,False,False,False,False,False,True


In [15]:
# barbJournal.dtypes

In [16]:
# barbJournal.excited.convert_objects(convert_numeric=True)

In [17]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [18]:
# indicator for walking
totalData = makeIndicators('excitement', barbMetrics, barbJournal, 0, 1, 18)
totalData = makeIndicators('happy', barbMetrics, barbJournal, 0, 1, 19)
totalData = makeIndicators('calm', barbMetrics, barbJournal, 0, 1, 20)
totalData = makeIndicators('anxious', barbMetrics, barbJournal, 0, 1, 21)
totalData = makeIndicators('sad', barbMetrics, barbJournal, 0, 1, 22)
totalData = makeIndicators('angry', barbMetrics, barbJournal, 0, 1, 23)
totalData = makeIndicators('hungry', barbMetrics, barbJournal, 0, 1, 24)
totalData = makeIndicators('tired', barbMetrics, barbJournal, 0, 1, 25)
totalData = makeIndicators('bored', barbMetrics, barbJournal, 0, 1, 26)

totalData = makeIndicators('exercise', barbMetrics, barbJournal, 0, 1, 36)
totalData = makeIndicators('code', barbMetrics, barbJournal, 0, 1, 37)
totalData = makeIndicators('anne', barbMetrics, barbJournal, 0, 1, 38)
totalData = makeIndicators('will', barbMetrics, barbJournal, 0, 1, 39)
totalData = makeIndicators('mick', barbMetrics, barbJournal, 0, 1, 40)
totalData = makeIndicators('geyer', barbMetrics, barbJournal, 0, 1, 41)
totalData = makeIndicators('home', barbMetrics, barbJournal, 0, 1, 42)
totalData = makeIndicators('work', barbMetrics, barbJournal, 0, 1, 43)
totalData = makeIndicators('school', barbMetrics, barbJournal, 0, 1, 44)


In [19]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  10080.000000  10080.000000  10080.000000  10080.000000  10080.000000   
mean      65.284623      3.466567      1.658363      0.001082     70.396171   
std       35.596657     16.000542      1.467958      0.018148     35.013143   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       57.000000      0.000000      1.100000      0.000055     82.400000   
50%       76.000000      0.000000      1.300000      0.000060     86.900000   
75%       88.000000      0.000000      1.900000      0.000073     88.700000   
max      198.000000    124.000000     17.200000      0.719000     97.700000   

            airtemp   excitement      happy      calm   anxious    ...      \
count  10080.000000        10080      10080     10080     10080    ...       
mean      67.822421  0.000595238  0.0570437  0.500198  0.338294    ...       
std       33.805046    0.0243914   0.231938  0.500025 

In [20]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm  ...    bored exercise   code   anne   will   mick  \
0      False  False  False  ...    False    False  False  False  False  False   
1      False  False  False  ...    False    False  False  False  False  False   
2      False  False  False  ...    False    False  False  False  False  False   
3      False  False  False  ...    False    False  False  False  False  False   
4      False  False  False  ...    False    False  False  False  False  False   

   geyer   home   work school  
0  False  False  F

In [21]:
totalData.to_csv('barbTotal.csv')

In [22]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,bored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 00:00:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-04-26 00:01:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-04-26 00:02:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-04-26 00:03:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-04-26 00:04:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(10080, 29)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm ...   will   mick  geyer   home   work school  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       1     4    0   0  
1       1     

In [24]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
code = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
anne = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
will = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
mick = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
geyer = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
school = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 28].values.reshape(totalData.shape[0], 1)

In [25]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
        exercise, 
        code, 
        anne, 
        will, 
        mick, 
        geyer, 
        home, 
        work, 
        school,
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(10080, 18)

In [26]:
# define y
y = totalData.excitement.values
y.shape

(10080,)

In [27]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [28]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [29]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [30]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [31]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000566893424036 0.000330687830688
      Accuracy Rate    0.999433106576    0.999669312169
     True Positives               0.4               0.0
     True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [32]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0189909297052 0.0198412698413
 Accuracy Rate  0.981009070295  0.980158730159
True Positives             0.0             0.0
True Negatives              --  0.980482963943


(0.01984126984126984, 0.98015873015873012, 0.0, 0.98048296394310286)

In [33]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000566893424036 0.000330687830688
 Accuracy Rate    0.999433106576    0.999669312169
True Positives               0.4               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [34]:
# define y
y = totalData.happy.values
y.shape

(10080,)

In [35]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [36]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [37]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0569727891156 0.0512566137566
 Accuracy Rate  0.943027210884  0.948743386243
True Positives 0.0289855072464 0.0372670807453
True Negatives              --             1.0


(0.051256613756613757, 0.94874338624338628, 0.037267080745341616, 1.0)

In [38]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0562641723356 0.0512566137566
      Accuracy Rate  0.943735827664  0.948743386243
     True Positives  0.048309178744 0.0434782608696
     True Negatives              --  0.999650716032


(0.051256613756613757,
 0.94874338624338628,
 0.043478260869565216,
 0.99965071603213418)

In [39]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.093112244898 0.0929232804233
 Accuracy Rate 0.906887755102  0.907076719577
True Positives 0.176328502415  0.136645962733
True Negatives             --  0.950401676563


(0.092923280423280422,
 0.90707671957671954,
 0.13664596273291926,
 0.95040167656304575)

In [40]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0571145124717 0.0512566137566
 Accuracy Rate  0.942885487528  0.948743386243
True Positives 0.0265700483092 0.0372670807453
True Negatives              --             1.0


(0.051256613756613757, 0.94874338624338628, 0.037267080745341616, 1.0)

In [41]:
# define y
y = totalData.calm.values
y.shape

(10080,)

In [42]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [43]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [44]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.265164399093 0.258267195767
 Accuracy Rate 0.734835600907 0.741732804233
True Positives 0.745521751493 0.775737704918
True Negatives             -- 0.707138092061


(0.25826719576719576,
 0.74173280423280419,
 0.77573770491803273,
 0.70713809206137423)

In [45]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.269699546485 0.260912698413
      Accuracy Rate 0.730300453515 0.739087301587
     True Positives 0.861814046062  0.87737704918
     True Negatives             -- 0.598398932622


(0.26091269841269843,
 0.73908730158730163,
 0.87737704918032788,
 0.59839893262174781)

In [46]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.294217687075 0.289682539683
 Accuracy Rate 0.705782312925 0.710317460317
True Positives 0.758316747228 0.773770491803
True Negatives             -- 0.645763842562


(0.28968253968253971,
 0.71031746031746024,
 0.77377049180327873,
 0.64576384256170782)

In [47]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.298185941043 0.292658730159
 Accuracy Rate 0.701814058957 0.707341269841
True Positives 0.988058003981 0.991475409836
True Negatives             -- 0.418278852568


(0.29265873015873017,
 0.70734126984126977,
 0.99147540983606552,
 0.41827885256837893)

In [48]:
# define y
y = totalData.anxious.values
y.shape

(10080,)

In [49]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [50]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [51]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set    Test Set  
-------------- -------------- ------------
    Error Rate 0.365646258503        0.375
 Accuracy Rate 0.634353741497        0.625
True Positives 0.807208717519 0.8056640625
True Negatives             --       0.5325


(0.375, 0.625, 0.8056640625, 0.53249999999999997)

In [52]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.281462585034 0.278439153439
      Accuracy Rate 0.718537414966 0.721560846561
     True Positives 0.240150880134    0.251953125
     True Negatives             --          0.962


(0.27843915343915343, 0.72156084656084651, 0.251953125, 0.96199999999999997)

In [53]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.367630385488 0.367394179894
 Accuracy Rate 0.632369614512 0.632605820106
True Positives 0.481559094719        0.46875
True Negatives             --         0.7165


(0.36739417989417988, 0.63260582010582012, 0.46875, 0.71650000000000003)

In [54]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.260487528345 0.256283068783
 Accuracy Rate 0.739512471655 0.743716931217
True Positives 0.237636211232   0.2529296875
True Negatives             --          0.995


(0.25628306878306878, 0.74371693121693117, 0.2529296875, 0.995)

In [55]:
# define y
y = totalData.sad.values
y.shape

(10080,)

In [56]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [57]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [58]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0668934240363 0.0658068783069
 Accuracy Rate   0.933106575964  0.934193121693
True Positives 0.00212765957447             0.0
True Negatives               --  0.999646142958


(0.065806878306878314, 0.93419312169312163, 0.0, 0.99964614295824483)

In [59]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0520124716553 0.0555555555556
      Accuracy Rate  0.947987528345  0.944444444444
     True Positives  0.946808510638  0.959595959596
     True Negatives              --  0.943382873319


(0.055555555555555552,
 0.94444444444444442,
 0.95959595959595956,
 0.94338287331917903)

In [60]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0761054421769 0.0816798941799
 Accuracy Rate  0.923894557823   0.91832010582
True Positives   0.56170212766  0.550505050505
True Negatives              --  0.944090587403


(0.081679894179894186,
 0.91832010582010581,
 0.5505050505050505,
 0.94409058740268936)

In [61]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0609410430839 0.0611772486772
 Accuracy Rate  0.939058956916  0.938822751323
True Positives  0.104255319149 0.0757575757576
True Negatives              --  0.999292285916


(0.061177248677248677,
 0.93882275132275128,
 0.07575757575757576,
 0.99929228591648978)

In [62]:
# define y
y = totalData.angry.values
y.shape

(10080,)

In [63]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [64]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [65]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.294075963719 0.281084656085
 Accuracy Rate 0.705924036281 0.718915343915
True Positives 0.721739130435 0.735677083333
True Negatives             -- 0.713209219858


(0.2810846560846561,
 0.71891534391534395,
 0.73567708333333337,
 0.713209219858156)

In [66]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.193452380952 0.166997354497
      Accuracy Rate 0.806547619048 0.833002645503
     True Positives 0.670108695652       0.703125
     True Negatives             -- 0.877216312057


(0.16699735449735451, 0.83300264550264547, 0.703125, 0.87721631205673756)

In [67]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.221088435374 0.194113756614
 Accuracy Rate 0.778911564626 0.805886243386
True Positives 0.655434782609 0.678385416667
True Negatives             -- 0.849290780142


(0.19411375661375663,
 0.80588624338624337,
 0.67838541666666663,
 0.849290780141844)

In [68]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.210884353741 0.191468253968
 Accuracy Rate 0.789115646259 0.808531746032
True Positives 0.379891304348 0.408854166667
True Negatives             -- 0.944592198582


(0.19146825396825398,
 0.80853174603174605,
 0.40885416666666669,
 0.94459219858156029)

In [69]:
# define y
y = totalData.hungry.values
y.shape

(10080,)

In [70]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [71]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [72]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.015306122449 0.0125661375661
 Accuracy Rate 0.984693877551  0.987433862434
True Positives 0.701149425287  0.730158730159
True Negatives             --   0.99861973775


(0.012566137566137565,
 0.98743386243386244,
 0.73015873015873012,
 0.99861973775017254)

In [73]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set   
------------------- ---------------- ---------------
         Error Rate 0.00184240362812 0.0026455026455
      Accuracy Rate   0.998157596372  0.997354497354
     True Positives   0.994252873563  0.992063492063
     True Negatives               --  0.997584541063


(0.0026455026455026454,
 0.99735449735449733,
 0.99206349206349209,
 0.99758454106280192)

In [74]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set     Test Set   
-------------- --------------- --------------
    Error Rate 0.0949546485261  0.10119047619
 Accuracy Rate  0.905045351474  0.89880952381
True Positives  0.261494252874 0.285714285714
True Negatives              -- 0.925465838509


(0.10119047619047619,
 0.89880952380952384,
 0.2857142857142857,
 0.92546583850931674)

In [75]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0488945578231 0.0396825396825
 Accuracy Rate  0.951105442177  0.960317460317
True Positives  0.014367816092  0.047619047619
True Negatives              --             1.0


(0.03968253968253968, 0.96031746031746035, 0.047619047619047616, 1.0)

In [76]:
# define y
y = totalData.tired.values
y.shape

(10080,)

In [77]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [78]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [79]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [80]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set        Test Set    
------------------- ---------------- -----------------
         Error Rate 0.00127551020408 0.000992063492063
      Accuracy Rate   0.998724489796    0.999007936508
     True Positives              0.0               0.0
     True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [81]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00751133786848 0.00925925925926
 Accuracy Rate   0.992488662132   0.990740740741
True Positives              0.0              0.0
True Negatives               --   0.991724594505


(0.0092592592592592587, 0.9907407407407407, 0.0, 0.99172459450513073)

In [82]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [83]:
# define y
y = totalData.bored.values
y.shape

(10080,)

In [84]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [85]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [86]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [87]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000708616780045 0.000330687830688
      Accuracy Rate     0.99929138322    0.999669312169
     True Positives               0.0               0.0
     True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [88]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0189909297052 0.0198412698413
 Accuracy Rate  0.981009070295  0.980158730159
True Positives             0.0             0.0
True Negatives              --  0.980482963943


(0.01984126984126984, 0.98015873015873012, 0.0, 0.98048296394310286)

In [89]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)